In [1]:
location = 'C:/_kopp/Dropbox/Filipe/_github/opengov-br/'

In [30]:
import urllib.request
from bs4 import BeautifulSoup

url     = 'http://idg.receita.fazenda.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-abertos-do-cnpj'
content = urllib.request.urlopen(url).read()
soup    = BeautifulSoup(content.decode('utf-8','ignore'),'html.parser')
table   = soup.find("table", attrs = {'class' : 'plain'})

for a in table.find_all('a', href=True):
    link=a['href']
    uf = link[-2:]
    print(uf, end=' ')
    f = open(location + "receita/cnpj_" + uf + ".csv","w+")
    g = open(location + "receita/socio_" + uf + ".csv","w+")
    f.write("uf;cnpj;razao\n")
    g.write("uf;cnpj;tipo;cpf;quadro;nome\n")
    response = urllib.request.urlopen(link).readlines()
    for line in response:
        try:
            line = str(line.strip().decode('latin1')).replace(';',',')
            if line[0:2] == '01':
                f.write(uf+";" + line[2:16]+";" + line[16:]+"\n")
            if line[0:2] == '02':
                g.write(uf+";" + line[2:16]+";" + line[16:17]+";" + line[17:31].strip() +";" + line[31:33]+";" + line[33:].strip()+"\n")
        except:
            print("erro: " + line)

RS 

In [36]:
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

url ='http://arquivos.portaldatransparencia.gov.br/PortalTransparenciaEscolheTipoDePlanilha.asp?origem=Servidores&Planilha=0&fechar=1'
with  urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        for name in zfile.namelist():
            zfile.extract(name,'./servidores')

ended in 211 seconds

Cadastro de Empresas Inidôneas e Suspensas (CEIS)
http://www.portaldatransparencia.gov.br/downloads/snapshot.asp?c=CEIS#get
http://arquivos.portaldatransparencia.gov.br/downloads.asp?a=2018&m=01&d=05&consulta=CEIS


obtain data from http://www.tse.jus.br/partidos/filiacao-partidaria/relacao-de-filiados


In [37]:
import urllib.request
from bs4 import BeautifulSoup
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

url = 'http://filiaweb.tse.jus.br/filiaweb/portal/relacoesFiliados.xhtml'
content = urllib.request.urlopen(url).read()
soup = BeautifulSoup(content.decode('utf-8','ignore'),'html.parser')
partidos = soup.find("select", {"id":"partido"})
ufs = soup.find("select", {"id":"uf"})
for uf in ufs.find_all('option', value=True):
    uf = uf['value']
    for partido in partidos.find_all('option', value=True):
        partido = partido['value']
        link = 'http://agencia.tse.jus.br/estatistica/sead/eleitorado/filiados/uf/filiados_'+partido+'_'+uf+'.zip'
        with urlopen(link) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                for name in zfile.namelist():
                    if "filiados_" in name:
                        zfile.extract(name,'./filiados')

In [2]:
import numpy as np
import pandas as pd

In [40]:
remuneracao = pd.read_csv(location + 'servidores/20130131_Remuneracao_Servidores.csv', sep=';', 
                         encoding='iso8859_2',  decimal=',') #skipfooter=2, nrows=100000,, engine='python'
remuneracao.to_pickle('remuneracao.pickle')
remuneracao.describe()

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,MES,ID_SERVIDOR_PORTAL,REMUNERAÇĂO BÁSICA BRUTA (R$),REMUNERAÇĂO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇĂO NATALINA (R$),GRATIFICAÇĂO NATALINA (U$),ABATE-TETO DA GRATIFICAÇĂO NATALINA (R$),ABATE-TETO DA GRATIFICAÇĂO NATALINA (U$),...,DEMAIS DEDUÇŐES (U$),REMUNERAÇĂO APÓS DEDUÇŐES OBRIGATÓRIAS (R$),REMUNERAÇĂO APÓS DEDUÇŐES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*),TOTAL DE JETONS
count,2.375831e+06,2.375831e+06,2.375831e+06,2.375831e+06,2.375831e+06,2375831.0,2.375831e+06,2.375831e+06,2.375831e+06,2375831.0,...,2.375831e+06,2.375831e+06,2.375831e+06,2.375831e+06,2.375831e+06,2.375831e+06,2375831.0,2.375831e+06,2.375831e+06,2.375831e+06
mean,8.998720e+00,1.521249e+06,5.719967e+03,6.296094e+00,-2.764178e+00,0.0,2.354953e+03,2.132811e-02,-8.436845e-01,0.0,...,-2.239927e-02,6.000559e+03,5.761916e+00,3.828163e+02,3.212134e+00,1.406332e+02,0.0,5.234495e+02,3.212134e+00,1.929542e+00
std,4.495029e+00,3.007805e+05,4.651842e+03,2.220757e+02,1.599740e+02,0.0,3.788002e+03,9.520499e+00,8.693305e+01,0.0,...,7.864151e+00,4.854495e+03,2.061779e+02,3.409584e+02,1.254376e+02,1.465013e+03,0.0,1.467984e+03,1.254376e+02,1.116696e+02
min,1.000000e+00,1.000000e+06,0.000000e+00,0.000000e+00,-3.431955e+04,0.0,-9.239220e+03,0.000000e+00,-2.672313e+04,0.0,...,-7.189320e+03,-4.578759e+04,-7.579500e+02,-4.956700e+02,0.000000e+00,0.000000e+00,0.0,-4.956700e+02,0.000000e+00,0.000000e+00
25%,1.100000e+01,1.260880e+06,2.629070e+03,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.000000e+00,2.789490e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2.646200e+02,0.000000e+00,0.000000e+00
50%,1.100000e+01,1.521534e+06,4.380620e+03,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.000000e+00,4.805880e+03,0.000000e+00,3.965100e+02,0.000000e+00,0.000000e+00,0.0,4.404000e+02,0.000000e+00,0.000000e+00
75%,1.200000e+01,1.781632e+06,7.631430e+03,0.000000e+00,0.000000e+00,0.0,3.620650e+03,0.000000e+00,0.000000e+00,0.0,...,0.000000e+00,7.772795e+03,0.000000e+00,5.733000e+02,0.000000e+00,0.000000e+00,0.0,5.980000e+02,0.000000e+00,0.000000e+00
max,1.200000e+01,2.042289e+06,6.237887e+04,1.823266e+04,0.000000e+00,0.0,5.954158e+04,7.330990e+03,2.168400e+03,0.0,...,0.000000e+00,1.176704e+05,2.406255e+04,7.286815e+04,1.552200e+04,8.574992e+04,0.0,8.574992e+04,1.552200e+04,3.417777e+04


In [41]:
servidores = pd.read_csv(location + 'servidores/20130131_Servidores.csv', sep='\t', 
                         encoding='iso8859_2', decimal=',') #nrows=100000, 

servidores.to_pickle('servidores.pickle')
servidores.tail()

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,...,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_SERVICOPUBLICO
1068069,1526949,KATIA REGINA DRUZIANI ROQUE,***.038.668-**,019****,NaN,NaN,NaN,NaN,NaN,DAS,...,Năo informada,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,28/11/2012,NaN,11/09/2012,310,11/09/2012,PORTARIA,PORTARIA
1068070,1404628,KATIA REGINA DUARTE,***.513.936-**,007****,AGENTE DE SERV COMPLEMENTARES,S,NaN,III,NaN,NaN,...,Năo informada,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,30/12/2006,NaN,21/10/1994,000000001,01/06/1987,PORTARIA,CONTRATO
1068071,1404628,KATIA REGINA DUARTE,***.513.936-**,007****,NaN,NaN,NaN,NaN,NaN,FGR,...,Năo informada,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/09/2006,NaN,21/10/1994,000000001,01/06/1987,PORTARIA,CONTRATO
1068072,1541698,KATIA REGINA ERN,***.636.159-**,013****,NaN,NaN,NaN,NaN,NaN,NaN,...,Năo informada,CONTRATO TEMPORARIO,40 HORAS SEMANAIS,NaN,NaN,26/06/2009,460,26/06/2009,CONTRATO,CONTRATO
1068073,2022770,KATIA REGINA ESTRELA GOMES,***.034.297-**,015****,TECNICO DE LABORATORIO,A,NaN,V,NaN,NaN,...,Năo informada,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,09/08/2007,NaN,25/07/2007,240,25/07/2007,PORTARIA,PORTARIA


In [3]:
import glob, os    

subdir   = location + 'filiados/aplic/sead/lista_filiados/uf/'
allFiles = glob.glob(subdir + "filiados_*.csv")
filiados = pd.DataFrame()
list_ = []
for file_ in allFiles:
    print('.', end='')
    df = pd.read_csv(file_,encoding='iso8859_2', header=0, sep=';')
    list_.append(df)
filiados = pd.concat(list_)
print(filiados.tail())
filiados.to_pickle('filiados.pickle')


C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_ac.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_al.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_am.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_ap.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_ba.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_ce.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_df.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_es.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_go.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/l

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_ms.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_mt.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_pa.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_avante_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/l

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_ro.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_rr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pcb_rs.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_p

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_ro.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_rr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_rs.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_sc.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pdt_se.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_p

...

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_ms.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_mt.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_pa.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psdb_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\f

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_ms.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_mt.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_pa.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_psl_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_p

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_ms.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_mt.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_pa.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_ptb_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_p

C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_pa.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_pb.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_pe.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_pi.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_pr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_rj.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_rn.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_ro.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_rr.csv
C:/_kopp/Dropbox/Filipe/_github/opengov-br/filiados/aplic/sead/lista_filiados/uf\filiados_pv_rs.csv


,DATA DA EXTRACAO,HORA DA EXTRACAO,NUMERO DA INSCRICAO,NOME DO FILIADO,SIGLA DO PARTIDO,NOME DO PARTIDO,UF,CODIGO DO MUNICIPIO,NOME DO MUNICIPIO,ZONA ELEITORAL,SECAO ELEITORAL,DATA DA FILIACAO,SITUACAO DO REGISTRO,TIPO DO REGISTRO,DATA DO PROCESSAMENTO,DATA DA DESFILIACAO,DATA DO CANCELAMENTO,DATA DA REGULARIZACAO,MOTIVO DO CANCELAMENTO
2676,01/02/2018,02:03:06,40377711074,DOUGLAS LAWSON GOMES PIMENTEL,SD,SOLIDARIEDADE,TO,73091,CAMPOS LINDOS,32,65,27/04/2016,REGULAR,OFICIAL,15/10/2016,NaN,NaN,NaN,NaN
2677,01/02/2018,02:03:06,26796081325,ANA MARIA LAGE RABELO,SD,SOLIDARIEDADE,TO,73440,PALMAS,29,241,17/11/2016,REGULAR,OFICIAL,18/04/2017,NaN,NaN,NaN,NaN
2678,01/02/2018,02:03:06,39239542739,LUCAS COELHO DOS SANTOS,SD,SOLIDARIEDADE,TO,96199,TOCANTÍNIA,5,37,16/06/2016,REGULAR,OFICIAL,18/04/2017,NaN,NaN,NaN,NaN
2679,01/02/2018,02:03:06,28961492780,CARLOS MAGNO REIS GOMES,SD,SOLIDARIEDADE,TO,95591,PORTO NACIONAL,3,115,17/02/2016,REGULAR,OFICIAL,18/04/2017,NaN,NaN,NaN,NaN
2680,01/02/2018,02:03:06,25416492798,ALCIR JOSÉ BORGES,SD,SOLIDARIEDADE,TO,73563,MONTE SANTO DO TOCANTINS,7,154,01/10/2013,REGULAR,OFICIAL,15/04/2016,NaN,NaN,NaN,NaN


In [31]:
allFiles = glob.glob(location + "receita/socio_*.csv")
socios   = pd.DataFrame()
list_    = []
for file_ in allFiles:
    df = pd.read_csv(file_,encoding='iso8859_2', header=0, sep=';') #, lineterminator="\n")
    list_.append(df)
socios = pd.concat(list_)
socios.to_pickle('socios.pickle')
socios.tail()

,uf,cnpj,tipo,cpf,quadro,nome
89906,TO,29237755000196,2,NaN,49,MARISTELA KUHN
89907,TO,29237755000196,2,NaN,49,CLEUDSON VICENTE DE SOUZA
89908,TO,29237802000100,2,NaN,16,EDINEI LOPES BORGES
89909,TO,29237916000141,2,NaN,65,ELDON MANOEL BARBOSA CARVALHO
89910,TO,29238000000106,2,NaN,65,SARAH BARREIROS MOTA


In [33]:
allFiles = glob.glob(location + "receita/cnpj_*.csv")
cnpj     = pd.DataFrame()
list_    = []
for file_ in allFiles:
    print('.', end='')
    df = pd.read_csv(file_,encoding='iso8859_2', header=0, sep=';')
    list_.append(df)
cnpj = pd.concat(list_)
cnpj.to_pickle('cnpj.pickle')
cnpj.tail()

...........................

,uf,cnpj,razao
51624,TO,29237691000123,ZE BENTO MAQUINAS E IMPLEMENTOS AGRICOLAS EIRE...
51625,TO,29237755000196,C. V. DE SOUZA E CIA LTDA - ME
51626,TO,29237802000100,ASSOCIACAO DE APOIO SUSTENTAVEIS AOS MUNICIPES...
51627,TO,29237916000141,REAL CONSULTORIA E AUDITORIA TRIBUTARIA - EIRE...
51628,TO,29238000000106,S B MOTA - EIRELI - ME


In [200]:
import urllib.request
from bs4 import BeautifulSoup
import requests

url = 'http://www.cnj.jus.br/transparencia/remuneracao-dos-magistrados'
content = urllib.request.urlopen(url).read()
soup = BeautifulSoup(content,'html.parser')
tribunal = soup.findAll("a", {"rel":"alternate noopener"})
for trib in tribunal:
    link = "http://www.cnj.jus.br"+ trib['href']
    ext = trib['href'].split('.',1)[1]
    print('.', end='')
    xls = requests.get(link)
    t = str(trib.text).strip()
    with open('./cnj/' + t + '.' + ext,'wb') as x:
        x.write(xls.content)

...............................................................................

In [206]:
import glob

allFiles    = glob.glob(location + "cnj/*.xls*")
tribunais   = pd.DataFrame()
list_       = []
field_names = ('cpf,nome,cargo,lotacao,subsidio,direitos_pessoais,'
                'indenizacoes,direitos_eventuais,total_de_rendimentos,'
                'previdencia_publica,imposto_de_renda,'
                'descontos_diversos,retencao_por_teto_constitucional,'
                'total_de_descontos,rendimento_liquido,'
                'remuneracao_do_orgao_de_origem,diarias').split(',')
for file_ in allFiles:
    print('.', end='')
    df = pd.read_excel(file_, header=None, usecols='A:Q', sheet_name='Contracheque', skiprows=20, names=field_names)
    df['filename'] = file_
    list_.append(df)
tribunais = pd.concat(list_)
tribunais = tribunais[pd.notnull(tribunais['nome'])]
tribunais = tribunais[tribunais.nome != 0]
for n in field_names:
    try: tribunais[n] = tribunais[n].str.strip()
    except: pass
tribunais.to_pickle('tribunais.pickle')
tribunais.to_csv('trib.csv')
tribunais.tail()

...............................................................................

,cpf,nome,cargo,lotacao,subsidio,direitos_pessoais,indenizacoes,direitos_eventuais,total_de_rendimentos,previdencia_publica,imposto_de_renda,descontos_diversos,retencao_por_teto_constitucional,total_de_descontos,rendimento_liquido,remuneracao_do_orgao_de_origem,diarias,filename
17,999.999.999-99,NAPOLEÃO NUNES MAIA FILHO,Ministro - Efetivo,Ministro 4 - Stj,0.0,0.0,0.0,5064.45,5064.45,0.00,523.36,0.00,0.0,523.36,4541.09,32074.85,0.0,C:/_kopp/Dropbox/Filipe/_github/opengov-br/cnj...
18,999.999.999-99,PAULO RICARDO DE SOUZA CRUZ,Juiz Convocado,Ministro 3 - Stj,3127.3,0.0,0.0,1042.43,4169.73,344.00,65.95,0.00,0.0,409.95,3759.78,28947.55,0.0,C:/_kopp/Dropbox/Filipe/_github/opengov-br/cnj...
19,999.999.999-99,ROSA MARIA WEBER CANDIOTA DA ROSA,Ministro - Efetivo,Ministro 1 - Stf,0.0,0.0,0.0,5064.45,5064.45,0.00,523.36,0.00,0.0,523.36,4541.09,33763.00,0.0,C:/_kopp/Dropbox/Filipe/_github/opengov-br/cnj...
20,999.999.999-99,SERGIO SILVEIRA BANHOS,Ministro - Substituto,Sem Lotação,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,861.0,C:/_kopp/Dropbox/Filipe/_github/opengov-br/cnj...
21,999.999.999-99,TARCISIO VIEIRA DE CARVALHO NETO,Ministro - Efetivo,Ministro 6 - Jurista,0.0,0.0,0.0,5064.45,5064.45,334.25,60.03,2025.78,0.0,2420.06,2644.39,0.00,861.0,C:/_kopp/Dropbox/Filipe/_github/opengov-br/cnj...


# Dados TSE

In [26]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

for year in range(2016,1999,-2):
    link  = 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_'+str(year)+'.zip'
    with urlopen(link) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            for name in zfile.namelist():
                if "consulta_" in name:
                    zfile.extract(name, location+'tse/'+str(year)+'/')
                    

In [27]:
import pandas as pd
import glob

ate2010 = ('data_geracao,hora_geracao,ano_eleicao,num_turno,descricao_eleicao,sigla_uf,'
          'sigla_ue,descricao_ue,codigo_cargo,descricao_cargo,nome_candidato,sequencial_candidato,'
          'numero_candidato,cpf_candidato,nome_urna_candidato,cod_situacao_candidatura,'
          'des_situacao_candidatura,numero_partido,sigla_partido,nome_partido,codigo_legenda,'
          'sigla_legenda,composicao_legenda,nome_legenda,codigo_ocupacao,descricao_ocupacao,'
          'data_nascimento,num_titulo_eleitoral_candidato,idade_data_eleicao,codigo_sexo,descricao_sexo,'
          'cod_grau_instrucao,descricao_cod_instrucao,codigo_estado_civil,descricao_estado_civil,'
          'codigo_nacionalidade,descricao_nacionalidade,sigla_uf_nascimento,codigo_municipio_nascimento,'
          'nome_municipio_nascimento,despesa_max_campanha,cod_sit_tot_turno,desc_sit_tot_turno')

em2012 =  ('data_geracao,hora_geracao,ano_eleicao,num_turno,descricao_eleicao,sigla_uf,'
          'sigla_ue,descricao_ue,codigo_cargo,descricao_cargo,nome_candidato,sequencial_candidato,'
          'numero_candidato,cpf_candidato,nome_urna_candidato,cod_situacao_candidatura,'
          'des_situacao_candidatura,numero_partido,sigla_partido,nome_partido,codigo_legenda,'
          'sigla_legenda,composicao_legenda,nome_legenda,codigo_ocupacao,descricao_ocupacao,'
          'data_nascimento,num_titulo_eleitoral_candidato,idade_data_eleicao,codigo_sexo,descricao_sexo,'
          'cod_grau_instrucao,descricao_cod_instrucao,codigo_estado_civil,descricao_estado_civil,'
          'codigo_nacionalidade,descricao_nacionalidade,sigla_uf_nascimento,codigo_municipio_nascimento,'
          'nome_municipio_nascimento,despesa_max_campanha,cod_sit_tot_turno,desc_sit_tot_turno,nmemail')

depois2014 = ('data_geracao,hora_geracao,ano_eleicao,num_turno,descricao_eleicao,sigla_uf,'
          'sigla_ue,descricao_ue,codigo_cargo,descricao_cargo,nome_candidato,sequencial_candidato,'
          'numero_candidato,cpf_candidato,nome_urna_candidato,cod_situacao_candidatura,'
          'des_situacao_candidatura,numero_partido,sigla_partido,nome_partido,codigo_legenda,'
          'sigla_legenda,composicao_legenda,nome_legenda,codigo_ocupacao,descricao_ocupacao,'
          'data_nascimento,num_titulo_eleitoral_candidato,idade_data_eleicao,codigo_sexo,descricao_sexo,'
          'cod_grau_instrucao,descricao_cod_instrucao,codigo_estado_civil,descricao_estado_civil,'
          'codigo_cor_raca,descricao_cor_raca,'
          'codigo_nacionalidade,descricao_nacionalidade,sigla_uf_nascimento,codigo_municipio_nascimento,'
          'nome_municipio_nascimento,despesa_max_campanha,cod_sit_tot_turno,desc_sit_tot_turno,nmemail')

for year in range(2016,1999,-2):
    if   year <= 2010: cabeca = ate2010.split(',')
    elif year == 2012: cabeca = em2012.split(',')
    elif year >= 2014: cabeca = depois2014.split(',')
    allFiles = glob.glob(location + "tse/"+str(year)+"/consulta_*.txt")
    candidatos = pd.DataFrame()
    list_    = []
    for file_ in allFiles:
        try:
            df = pd.read_csv(file_,encoding='latin1', header=None, names=cabeca, sep=';', quotechar='"')
            df['filename'] = file_
            list_.append(df)
        except:
            print(file_)
    candidatos = pd.concat(list_)
    candidatos.to_pickle(location + 'candidatos_' + str(year) + '.pickle')
    candidatos.to_csv(location + 'tse/candidatos'+ str(year) + '.csv')
candidatos.head()
candidatos.tail()



C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15,17,20,24,29,31,33,35,38,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13,15,17,20,24,29,31,33,35,38,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=r

,data_geracao,hora_geracao,ano_eleicao,num_turno,descricao_eleicao,sigla_uf,sigla_ue,descricao_ue,codigo_cargo,descricao_cargo,...,descricao_estado_civil,codigo_nacionalidade,descricao_nacionalidade,sigla_uf_nascimento,codigo_municipio_nascimento,nome_municipio_nascimento,despesa_max_campanha,cod_sit_tot_turno,desc_sit_tot_turno,filename
21813,25/11/2016,18:25:34,2000,2,ELEICOES 2000,TO,96431,XAMBIOA,13,VEREADOR,...,CASADO(A),1,BRASILEIRA,TO,-1,ARAGUATINS,-1.0,NaN,NaN,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
21814,25/11/2016,18:25:34,2000,2,ELEICOES 2000,TO,96431,XAMBIOA,13,VEREADOR,...,CASADO(A),1,BRASILEIRA,TO,-1,ARAGUATINS,-1.0,NaN,NaN,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
21815,25/11/2016,18:25:34,2000,2,ELEICOES 2000,TO,96431,XAMBIOA,13,VEREADOR,...,CASADO(A),1,BRASILEIRA,TO,-1,ARAGUATINS,-1.0,NaN,NaN,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
21816,25/11/2016,18:25:34,2000,2,ELEICOES 2000,TO,96431,XAMBIOA,13,VEREADOR,...,CASADO(A),1,BRASILEIRA,MA,-1,RIBEIRO GONÇALVES,-1.0,NaN,NaN,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
21817,25/11/2016,18:25:34,2000,2,ELEICOES 2000,TO,96431,XAMBIOA,13,VEREADOR,...,CASADO(A),1,BRASILEIRA,MA,-1,RIBEIRO GONÇALVES,-1.0,NaN,NaN,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...


In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

for year in range(2016,2005,-2):
    link  = 'http://agencia.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_'+str(year)+'.zip'
    with urlopen(link) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            for name in zfile.namelist():
                if "bem_" in name:
                    zfile.extract(name, location+'tse/'+str(year)+'/')
                    

In [36]:
import pandas as pd
import glob

cabeca = ('data_geracao,hora_geracao,ano_eleicao,descricao_eleicao,sigla_uf,'
          'sequencial_candidato,cd_tipo_bem_candidato,ds_tipo_bem_candidato,'
          'detalhe_bem,valor_bem,data_ultima_atulizacao,hora_ultima_atualizacao').split(',')

for year in range(2016,2005,-2):
    print('\n'+str(year))
    allFiles = glob.glob(location + "tse/"+str(year)+"/bem_*.txt")
    bem = pd.DataFrame()
    list_    = []
    for file_ in allFiles:
        try:
            df = pd.read_csv(file_,encoding='latin1', header=None, names=cabeca, sep=';', quotechar='"')
            df['filename'] = file_
            list_.append(df)
        except:
            print(file_)
    bem = pd.concat(list_)
    bem.to_pickle(location + 'bem_' + str(year) + '.pickle')
    bem.to_csv(location + 'tse/bem'+ str(year) + '.csv')
bem.head()
bem.tail()


2016

2014

2012

2010

2008

2006


,data_geracao,hora_geracao,ano_eleicao,descricao_eleicao,sigla_uf,sequencial_candidato,cd_tipo_bem_candidato,ds_tipo_bem_candidato,detalhe_bem,valor_bem,data_ultima_atulizacao,hora_ultima_atualizacao,filename
1849,26/02/2013,16:23:42,2006,ELEICOES 2006,TO,10760,-1,#NE#,LOTE 03A/02 LOTEAMENTO PORTEIRA EM PORTO NACIO...,2000.00,#NE#,#NE#,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
1850,26/02/2013,16:23:42,2006,ELEICOES 2006,TO,10760,-1,#NE#,"LOTE 06 QD ARSO 43 CONJ QI 3 ALAMEDA 12, COM A...",6633.62,#NE#,#NE#,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
1851,26/02/2013,16:23:42,2006,ELEICOES 2006,TO,10760,-1,#NE#,"LOTE 09 QD ACSUSO 60 CONJ 01 PALMAS - TO,ADQ D...",27000.00,#NE#,#NE#,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
1852,26/02/2013,16:23:42,2006,ELEICOES 2006,TO,10760,-1,#NE#,"LOTE 18 QD ARSO 34, CONJ. QI 28 RUA 02, COM AR...",8109.91,#NE#,#NE#,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...
1853,26/02/2013,16:23:42,2006,ELEICOES 2006,TO,10760,-1,#NE#,SALDO BANCARIO NO BASA SA,641.12,#NE#,#NE#,C:/_kopp/Dropbox/Filipe/_github/opengov-br/tse...


In [34]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

ufs = ('BR,VT,AP,AM,AC,SC,RJ,SP,TO,PB,PR,RS,RN,RR,RO,ES,MG,BA,SE,PB,MA,PE,PA,MS,MT,DF,GO,CE,AL,PI').split(',')
for year in range(2016,1999,-2):
    print('\n'+ str(year), end=' ')
    for uf in ufs:
        print(uf,end='')
        link  = 'http://agencia.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_'+str(year)+'_'+uf+'.zip'
        try:
            with urlopen(link) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    for name in zfile.namelist():
                        if "votacao_" in name:
                            zfile.extract(name, location+'tse/'+str(year)+'/')
            print('.',end=' ')
        except:
            print('*', end=' ')




2016 BR* VT* AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF* GO. CE. AL. PI. 
2014 BR. VT* AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. AL. PI. 
2012 BR* VT* AP* AM* AC* SC* RJ* SP* TO* PB* PR* RS* RN* RR* RO* ES* MG* BA* SE* PB* MA* PE* PA* MS* MT* DF* GO* CE* AL* PI* 
2010 BR. VT. AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. AL. PI. 
2008 BR. VT. AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. AL. PI. 
2006 BR. VT. AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. AL. PI. 
2004 BR. VT. AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. AL. PI. 
2002 BR. VT. AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF. GO. CE. 

In [35]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

year = 2012
for turno in range(1,3):
    if turno == 2:
        comp = '_30102012194527'
    else:
        comp = ''
        
    print('\n'+ str(year), end=' ')
    for uf in ufs:
        print(uf,end='')
        link  = 'http://agencia.tse.jus.br/estatistica/sead/eleicoes/eleicoes2012/votosecao/vsec_'+str(turno)+'t_'+uf+comp+'.zip'
        try:
            with urlopen(link) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    for name in zfile.namelist():
                        if "votacao_" in name:
                            zfile.extract(name, location+'tse/'+str(year)+'/')
            print('.',end=' ')
        except:
            print('*', end=' ')


2012 BR* VT* AP. AM. AC. SC. RJ. SP. TO. PB. PR. RS. RN. RR. RO. ES. MG. BA. SE. PB. MA. PE. PA. MS. MT. DF* GO. CE. AL. PI. 
2012 BR* VT* AP. AM. AC. SC. RJ. SP. TO* PB. PR. RS. RN. RR* RO. ES. MG. BA. SE* PB. MA. PE* PA. MS. MT. DF* GO* CE. AL* PI. 

In [2]:
import pandas as pd
import glob

location = 'C:/_kopp/Dropbox/Filipe/_github/opengov-br/'
cabeca = ('data_geracao,hora_geracao,ano_eleicao,num_turno,descricao_eleicao,'
          'sigla_uf,sigla_ue,codigo_municipio,nome_municipio,num_zona,num_secao,'
          'codigo_cargo,descricao_cargo,num_votavel,qtde_votos').split(',')

for year in range(2016,1999,-2):
    print('\n'+str(year))
    allFiles = glob.glob(location + "tse/"+str(year)+"/votacao_*.txt")
    votacao = pd.DataFrame()
    list_    = []
    for file_ in allFiles:
        try:
            df = pd.read_csv(file_,encoding='latin1', header=None, names=cabeca, sep=';', quotechar='"')
            df['filename'] = file_
            list_.append(df)
        except:
            print(file_)
    votacao = pd.concat(list_)
    votacao.to_pickle(location + 'votacao_' + str(year) + '.pickle')
    votacao.to_csv(location + 'tse/votacao'+ str(year) + '.csv')
votacao.head()
votacao.tail()


2016


MemoryError: 